
# SixtyFour AI — Find Phone (Beta)

This notebook is your guide to using the Sixtyfour API `find-phone` (beta).

The `find-phone` endpoint discovers phone numbers for a lead from the data you provide. It looks across relevant sources and returns a phone number as a string or an array of phone objects (number + region).

**Tip:** You can change the lead in the parameters at the top. Include **name**, **company**, and a strong identifier (e.g., **LinkedIn**, **domain**, or **email**) for best results.



## Prerequisites & Secrets

### Get your SixtyFour API key
1) Sign in: https://app.sixtyfour.ai/login  
2) Create an organization → **Keys** → **Create new key** → Copy the key.

### If you’re in Google Colab
- Click the 🔑 (Secrets) icon in the left sidebar.  
- Add `SIXTYFOUR_API_KEY` and paste your key.  
- Make sure the notebook can access the secret.


In [1]:

# --- Parameters (you can change these) ---
LEAD_NAME = "Saarth Shah"
LEAD_COMPANY = "Sixtyfour AI"
# LEAD_LINKEDIN = "https://linkedin.com/in/sarahjohnson"
# LEAD_DOMAIN = None
# LEAD_EMAIL = None

import os, json, requests

API_KEY = os.getenv("SIXTYFOUR_API_KEY", "")
try:
    from google.colab import userdata
    secret_val = userdata.get("SIXTYFOUR_API_KEY")
    if secret_val:
        os.environ["SIXTYFOUR_API_KEY"] = secret_val
        API_KEY = secret_val
except Exception:
    pass

BASE_URL = "https://api.sixtyfour.ai"
ENDPOINT = f"{BASE_URL}/find-phone"
TIMEOUT_S = 180

print("✅ Setup complete (key detected, endpoint ready).")


✅ Setup complete (key detected, endpoint ready).



## Find_Phone Request

For most use cases, `find-phone` is straightforward: pass a `lead` object and receive a phone value in return.

The request body requires:

- **lead (object)**: Lead information (name/company recommended; LinkedIn/domain/email help).

Responses may return:
- **String**: `"+1 555-123-4567"`  
- **Array of objects**: `[{"number": "+1 555-123-4567", "region": "US"}, ...]`  
- **Empty string**: `""` when no phone is found.


In [3]:

lead = {
    "name": LEAD_NAME,
    "company": LEAD_COMPANY,
}
# if LEAD_LINKEDIN: lead["linkedin_url"] = LEAD_LINKEDIN
# if LEAD_DOMAIN: lead["domain"] = LEAD_DOMAIN
# if LEAD_EMAIL: lead["email"] = LEAD_EMAIL

payload = {"lead": lead}

resp = requests.post(
    ENDPOINT,
    headers={"x-api-key": API_KEY, "Content-Type": "application/json"},
    json=payload,
    timeout=TIMEOUT_S,
)

if resp.ok:
    data = resp.json()
    print("✅ Success! Keys:", list(data.keys()))
    print("\nRaw response:")
    print(json.dumps(data, indent=2))

    # Handle string vs array formats
    phone_val = data.get("phone", "")
    print("\nParsed phone(s):")
    if isinstance(phone_val, str):
        print(" -", phone_val or "Not found")
    elif isinstance(phone_val, list):
        for p in phone_val:
            if isinstance(p, dict):
                print(f" - {p.get('number')} (region={p.get('region')})")
            else:
                print(" -", p)
    else:
        print(" - Not found")
else:
    print("❌ Request failed:", resp.status_code, resp.text[:800])


✅ Success! Keys: ['name', 'company', 'phone']

Raw response:
{
  "name": "Saarth Shah",
  "company": "Sixtyfour AI",
  "phone": "+1 502-388-0975"
}

Parsed phone(s):
 - +1 502-388-0975



### How to change values

- Edit the lead fields (name/company/LinkedIn/domain/email).  
- If you have multiple identifiers, include them—more context improves accuracy.  
- Expect `phone` to be a **string**, an **array of objects**, or an **empty string**.
